In [20]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [45]:
CLIENT_ID = 'RM24NDAJSW52KGHMEUPAOMKEE1VVLBNO1OCV4SUJ4EKVXFNS' 
# your Foursquare ID
CLIENT_SECRET = 'KVR4B32R0H3JGH31BCCPK30REKHYPMSKJXM1ILNAKFDWVTXN' 
# your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: RM24NDAJSW52KGHMEUPAOMKEE1VVLBNO1OCV4SUJ4EKVXFNS
CLIENT_SECRET:KVR4B32R0H3JGH31BCCPK30REKHYPMSKJXM1ILNAKFDWVTXN


In [21]:
from operator import itemgetter
from itertools import chain

##### Reference
https://medium.com/analytics-vidhya/web-scraping-wiki-tables-using-beautifulsoup-and-python-6b9ea26d8722

In [22]:
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

from bs4 import BeautifulSoup
# extract the whole website
soup = BeautifulSoup(website_url,'lxml')
# print(soup.prettify())

In [23]:
# extract the whole table
My_table = soup.find('table',{'class':'wikitable sortable'})

In [24]:
# extract all things about postcodes, borough and neighborhood
lists = My_table.findAll('td')
newlists = []
for item in lists:
    tmpstr = str(item)
    if tmpstr.find('</a>') == -1:
        newlists.append(tmpstr[tmpstr.find('<td>')+4:tmpstr.find('</td>')].replace('\n',''))
    else:
        newlists.append(tmpstr[tmpstr.find('">')+2:tmpstr.find('</a>')].replace('\n',''))
        
newlists[:6]

['M1A', 'Not assigned', 'Not assigned', 'M2A', 'Not assigned', 'Not assigned']

In [25]:
# extract columns
postcodes = newlists[0::3]
borough = newlists[1::3]
neighborhood = newlists[2::3]

In [26]:
neighborhood[-3:]

['Royal York South West', 'South of Bloor', 'Not assigned']

In [27]:
p,b,n = [],[],[]
for i in range(len(postcodes)):
    if borough[i] != 'Not assigned':
        p.append(postcodes[i])
        b.append(borough[i])
        if neighborhood[i] == 'Not assigned':
            n.append(borough[i])
        else:
            n.append(neighborhood[i])

In [28]:
from operator import itemgetter

In [29]:
POST,BOR,NEIGH = [],[],[]
for i in range(len(p)):
    if p[i] not in POST:
        POST.append(p[i])
        BOR.append(b[i])
        tmpid = [j for j,x in enumerate(p) if x==p[i]]
        tmgetter = itemgetter(*tmpid)
        tmplist = tmgetter(n)
        if isinstance(tmplist,str):
            NEIGH.append(tmplist)
        elif isinstance(tmplist,tuple):
            NEIGH.append(','.join(list(tmplist)))

In [30]:
df = pd.DataFrame()
df['PostalCode'] = POST
df['Borough'] = BOR
df['Neighborhood'] = NEIGH

#### Geographical coordinates of each postal code

#### Read csv

In [31]:
spadata = pd.read_csv('http://cocl.us/Geospatial_data')

#### Rename the column Postal Code

In [32]:
spadata.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)

#### Merge two dataframes

In [33]:
df2 = pd.merge(df,spadata)

# Toronto neighborhoods

#### work with only boroughs that contain the word Toronto

In [34]:
df3 = df2[df2['Borough'].str.contains("Toronto")]
df3.shape

(38, 5)

In [35]:
print('The dataframe has {} boroughs.'.format(
        len(df3['Borough'].unique())))

The dataframe has 4 boroughs.


In [36]:
address = 'Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [37]:
nei = df3.columns.difference(['Neighborhood'])
genres = df3['Neighborhood'].str.split(',')

df4 =  (df3.loc[df3.index.repeat(genres.str.len()),nei]
         .assign(Neighborhood=list(chain.from_iterable(genres.tolist())))).reset_index(drop=True)

In [38]:
df4.head()

,Borough,Latitude,Longitude,PostalCode,Neighborhood
0,Downtown Toronto,43.654260,-79.360636,M5A,Harbourfront
1,Downtown Toronto,43.654260,-79.360636,M5A,Regent Park
2,Downtown Toronto,43.657162,-79.378937,M5B,Ryerson
3,Downtown Toronto,43.657162,-79.378937,M5B,Garden District
4,Downtown Toronto,43.651494,-79.375418,M5C,St. James Town


In [39]:
df4.shape

(74, 5)

In [40]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df4['Borough'].unique()),
        df4.shape[0]))

The dataframe has 4 boroughs and 74 neighborhoods.


In [41]:
# create map of New York using latitude and longitude values
map_tor = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df4['Latitude'],df4['Longitude'],df4['Borough'],df4['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_tor)  
    
map_tor

#### Let's explore the first neighborhood in our dataframe.

In [42]:
df4.loc[0,'Neighborhood']

'Harbourfront'

##### So Let's explore Harbourfront

In [43]:
neighborhood_latitude = df4.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df4.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = df4.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Harbourfront are 43.6542599, -79.3606359.


In [46]:
radius = 500
LIMIT = 30
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

In [47]:
results = requests.get(url).json()

In [48]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [49]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Roselle Desserts,Bakery,43.653447,-79.362017
1,Tandem Coffee,Coffee Shop,43.653559,-79.361809
2,Toronto Cooper Koo Family Cherry St YMCA Centre,Gym / Fitness Center,43.653191,-79.357947
3,Body Blitz Spa East,Spa,43.654735,-79.359874
4,Morning Glory Cafe,Breakfast Spot,43.653947,-79.361149


In [50]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

30 venues were returned by Foursquare.


## 2. Explore Neighborhoods in Toronto

In [51]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [52]:
toronto_venues = getNearbyVenues(names=df4['Neighborhood'],
                                   latitudes=df4['Latitude'],
                                   longitudes=df4['Longitude']
                                  )

Harbourfront
Regent Park
Ryerson
Garden District
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Adelaide
King
Richmond
Dovercourt Village
Dufferin
Harbourfront East
Toronto Islands
Union Station
Little Portugal
Trinity
The Danforth West
Riverdale
Design Exchange
Toronto Dominion Centre
Brockton
Exhibition Place
Parkdale Village
The Beaches West
India Bazaar
Commerce Court
Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North
Forest Hill West
High Park
The Junction South
North Toronto West
The Annex
North Midtown
Yorkville
Parkdale
Roncesvalles
Davisville
Harbord
University of Toronto
Runnymede
Swansea
Moore Park
Summerhill East
Chinatown
Grange Park
Kensington Market
Deer Park
Forest Hill SE
Rathnelly
South Hill
Summerhill West
CN Tower
Bathurst Quay
Island airport
Harbourfront West
King and Spadina
Railway Lands
South Niagara
Rosedale
Stn A PO Boxes 25 The Esplanade
Cabbagetown
St. James Town
First Canadian Place
Underground city


In [53]:
print(toronto_venues.shape)
toronto_venues.head()

(1631, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Harbourfront,43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,Harbourfront,43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,Harbourfront,43.65426,-79.360636,Toronto Cooper Koo Family Cherry St YMCA Centre,43.653191,-79.357947,Gym / Fitness Center
3,Harbourfront,43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,Harbourfront,43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot


In [70]:

tmp2 = toronto_venues.groupby('Neighborhood').count()
tmp2.sort_values('Venue Category', inplace=True, ascending=False)
locate,count2 = [],[]
for i in range(30):
    locate.append(tmp2.index[i])
    count2.append(tmp2['Venue Category'][i])
Location = pd.DataFrame()
Location['Location'] = locate
Location['No. of shops'] = count2
Location

,Location,No. of shops
0,St. James Town,60
1,Adelaide,30
2,Grange Park,30
3,Harbourfront,30
4,Harbourfront East,30
5,Kensington Market,30
6,King,30
7,Regent Park,30
8,Richmond,30
9,Riverdale,30


# Week 5

In [66]:
tmp = toronto_venues.groupby('Venue Category').count()
tmp.sort_values('Neighborhood', inplace=True, ascending=False)
tmp['Neighborhood']

Venue Category
Café                             108
Coffee Shop                      103
Pizza Place                       44
Bakery                            40
Italian Restaurant                40
Park                              38
Bar                               37
Restaurant                        36
Pub                               34
Greek Restaurant                  25
Sandwich Place                    24
Japanese Restaurant               24
Deli / Bodega                     23
Gastropub                         23
American Restaurant               22
Steakhouse                        21
Sushi Restaurant                  21
Airport Service                   21
Breakfast Spot                    20
Hotel                             19
Vegetarian / Vegan Restaurant     19
Gym                               19
Liquor Store                      18
Mexican Restaurant                18
Dessert Shop                      17
Vietnamese Restaurant             16
Asian Restaurant       

In [67]:
# Find the top 10 shop types
shop,count = [],[]
for i in range(15):
    shop.append(tmp.index[i])
    count.append(tmp['Neighborhood'][i])

In [68]:
Venue = pd.DataFrame()
Venue['Venue Category'] = shop
Venue['No. of neighborhoods'] = count
Venue

,Venue Category,No. of neighborhoods
0,Café,108
1,Coffee Shop,103
2,Pizza Place,44
3,Bakery,40
4,Italian Restaurant,40
5,Park,38
6,Bar,37
7,Restaurant,36
8,Pub,34
9,Greek Restaurant,25


In [55]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 193 uniques categories.


## 3. Analyze Each Neighborhood

In [56]:
# one hot encoding
tor_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
tor_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [tor_onehot.columns[-1]] + list(tor_onehot.columns[:-1])
tor_onehot = tor_onehot[fixed_columns]

tor_onehot.head()

,Yoga Studio,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Dance Studio,Deli / Bodega,Dessert Shop,Diner,Discount Store,Dog Run,Dumpling Restaurant,Eastern European Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hotel,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Organic Grocery,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Harbourfront,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Harbourfront,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Harbourfront,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [57]:
tor_onehot.shape

(1631, 193)

In [58]:
tor_grouped = tor_onehot.groupby('Neighborhood').mean().reset_index()
tor_grouped.head()

,Neighborhood,Yoga Studio,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Dance Studio,Deli / Bodega,Dessert Shop,Diner,Discount Store,Dog Run,Dumpling Restaurant,Eastern European Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hotel,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Museum,Music Store,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Organic Grocery,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,Adelaide,0.0000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.033333,0.0,0.0,0.0,0.0,0.0,0.066667,0.0,0.0000,0.0,0.0,0.0,0.000000,0.0,0.033333,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.033333,0.0000,0.0,0.0,0.000000,0.0,0.0000,0.066667,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.033333,0.0,0.0,0.0,0.0000,0.033333,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0000,0.0,0.000000,0.0,0.0,0.0,0.0,0.033333,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0000,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.033333,0.000000,0.000000,0.033333,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.066667,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.000000,0.033333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.033333,0.0,0.000000,0.0,0.0,0.0,0.0,0.033333,0.0,0.033333,0.0,0.000000,0.000000,0.000000,0.0,0.066667,0.000000,0.0,0.033333,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.033333,0.0,0.0000,0.0,0.033333,0.0,0.0000,0.033333,0.0,0.0,0.000000,0.0,0.100000,0.0,0.033333,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.033333,0.0,0.0,0.0,0.0
1,Bathurst Quay,0.0000,0.0,0.066667,0.066667,0.066667,0.133333,0.2,0.133333,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,

#### Let's print each neighborhood along with the top 5 most common venues

In [59]:
num_top_venues = 5

for hood in tor_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = tor_grouped[tor_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide----
              venue  freq
0        Steakhouse  0.10
1  Asian Restaurant  0.07
2              Café  0.07
3       Pizza Place  0.07
4             Hotel  0.07


----Bathurst Quay----
              venue  freq
0   Airport Service  0.20
1  Airport Terminal  0.13
2    Airport Lounge  0.13
3   Harbor / Marina  0.07
4     Boat or Ferry  0.07


----Berczy Park----
                venue  freq
0        Cocktail Bar  0.07
1                Café  0.07
2  Seafood Restaurant  0.07
3      Farmers Market  0.07
4            Beer Bar  0.07


----Brockton----
                    venue  freq
0             Coffee Shop  0.11
1                    Café  0.11
2          Breakfast Spot  0.11
3           Grocery Store  0.05
4  Furniture / Home Store  0.05


----Business Reply Mail Processing Centre 969 Eastern----
           venue  freq
0    Yoga Studio  0.06
1  Auto Workshop  0.06
2     Comic Shop  0.06
3           Park  0.06
4     Restaurant  0.06


----CN Tower----
              venue  freq
0  

In [60]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [61]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = tor_grouped['Neighborhood']

for ind in np.arange(tor_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(tor_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adelaide,Steakhouse,Asian Restaurant,Hotel,Café,Pizza Place,Greek Restaurant,Breakfast Spot,Seafood Restaurant,Coffee Shop,Smoke Shop
1,Bathurst Quay,Airport Service,Airport Terminal,Airport Lounge,Boat or Ferry,Boutique,Plane,Harbor / Marina,Sculpture Garden,Airport Gate,Airport Food Court
2,Berczy Park,Beer Bar,Farmers Market,Cocktail Bar,Café,Seafood Restaurant,Bistro,Italian Restaurant,Bakery,Jazz Club,Basketball Stadium
3,Brockton,Coffee Shop,Café,Breakfast Spot,Italian Restaurant,Performing Arts Venue,Climbing Gym,Convenience Store,Caribbean Restaurant,Burrito Place,Restaurant
4,Business Reply Mail Processing Centre 969 Eastern,Yoga Studio,Auto Workshop,Light Rail Station,Spa,Brewery,Farmers Market,Fast Food Restaurant,Restaurant,Burrito Place,Butcher


## 4. Cluster Neighborhoods

In [62]:
# set number of clusters
kclusters = 5

tor_grouped_clustering = tor_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(tor_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 2, 0, 0, 0, 2, 0, 0, 0, 0], dtype=int32)

In [63]:
# add clustering labels
#neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

tor_merged = df4

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
tor_merged = tor_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

tor_merged.head() # check the last columns!

,Borough,Latitude,Longitude,PostalCode,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,43.654260,-79.360636,M5A,Harbourfront,Coffee Shop,Park,Bakery,Mexican Restaurant,Breakfast Spot,Café,Restaurant,Pub,Chocolate Shop,Yoga Studio
1,Downtown Toronto,43.654260,-79.360636,M5A,Regent Park,Coffee Shop,Park,Bakery,Mexican Restaurant,Breakfast Spot,Café,Restaurant,Pub,Chocolate Shop,Yoga Studio
2,Downtown Toronto,43.657162,-79.378937,M5B,Ryerson,Café,Clothing Store,Movie Theater,Art Gallery,Ramen Restaurant,Japanese Restaurant,Sandwich Place,Beer Bar,Coffee Shop,Plaza
3,Downtown Toronto,43.657162,-79.378937,M5B,Garden District,Café,Clothing Store,Movie Theater,Art Gallery,Ramen Restaurant,Japanese Restaurant,Sandwich Place,Beer Bar,Coffee Shop,Plaza
4,Downtown Toronto,43.651494,-79.375418,M5C,St. James Town,Restaurant,Coffee Shop,Gastropub,Italian Restaurant,Café,Japanese Restaurant,Gym,Hotel,Diner,Thai Restaurant


In [64]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(tor_merged['Latitude'],tor_merged['Longitude'],tor_merged['Neighborhood'],tor_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

KeyError: 'Cluster Labels'

## 5. Examine Clusters

#### cluster 1

In [ ]:
tor_merged.loc[tor_merged['Cluster Labels'] == 0,tor_merged.columns[[1] + list(range(5,tor_merged.shape[1]))]]

#### cluster 2

In [ ]:
tor_merged.loc[tor_merged['Cluster Labels'] == 1,tor_merged.columns[[1] + list(range(5,tor_merged.shape[1]))]]

#### cluster 3

In [ ]:
tor_merged.loc[tor_merged['Cluster Labels'] == 2,tor_merged.columns[[1] + list(range(5,tor_merged.shape[1]))]]

#### cluster 4

In [ ]:
tor_merged.loc[tor_merged['Cluster Labels'] == 3,tor_merged.columns[[1] + list(range(5,tor_merged.shape[1]))]]

#### cluster 5

In [ ]:
tor_merged.loc[tor_merged['Cluster Labels'] == 4,tor_merged.columns[[1] + list(range(5,tor_merged.shape[1]))]]